<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/11_language_modeling_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformerを使った言語モデル

* 参考資料: 言語モデルに関するPyTorchのチュートリアル
  * https://pytorch.org/tutorials/beginner/transformer_tutorial.html

## 今回の課題設定
* 英語のテキストのembeddingを得るために、トランスフォーマを一からtrainingする。

**ランタイムのタイプをGPUにしておく。**

## 準備

* 必要なライブラリのインストール
  * あとでデータセットを取得するために必要なライブラリ。

In [ ]:
!pip install 'portalocker>=2.0.0'

### `transformers`のインストール
* `transformers`はtokenizerを用意するために使う。
  * 今回は、モデルを作るために`transformers`は使わない。

In [ ]:
!pip install transformers

In [ ]:
import os
import time
import math
import torch
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

## モデルの定義




### Transformer encoderモデル

* PyTorchの`nn.TransformerEncoder`を使う
  * https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html
* デフォルトの設定で``batch_first=False``になっていることに注意。

In [ ]:
class TransformerModel(nn.Module):

  def __init__(self, ntoken, d_model, nhead, d_hid, nlayers, dropout=0.5):
    super().__init__()
    # 入力されるベクトルの次元（今回はtoken embeddingの次元）
    self.d_model = d_model
    # 位置エンコーディング
    self.pos_encoder = PositionalEncoding(d_model, dropout)
    # 多層のエンコーダを作成
    encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
    self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
    # 入力の埋め込み層
    self.encoder = nn.Embedding(ntoken, d_model)
    # 単語ロジットを出力する全結合層（ntokenは語彙サイズ）
    self.decoder = nn.Linear(d_model, ntoken)
    # 今回は、自前の初期化を使ってみる
    self.init_weights()

  def init_weights(self):
    initrange = 0.1
    self.encoder.weight.data.uniform_(- initrange, initrange)
    self.decoder.bias.data.zero_()
    self.decoder.weight.data.uniform_(- initrange, initrange)

  def forward(self, src, src_mask):
    src = self.encoder(src) * math.sqrt(self.d_model)
    src = self.pos_encoder(src)
    output = self.transformer_encoder(src, src_mask)
    output = self.decoder(output)
    return output

## 上三角行列のマスク
* 言語モデルは、次のトークンを予測するモデル。
* よって、過去のトークンだけを見るようにしないといけない。
* そのため、self-attentionの計算に、上三角行列のマスクをかける。

In [ ]:
def generate_square_subsequent_mask(mask_size):
  #上三角行列を生成する。上三角が-inf、対角成分含めた残りはゼロ。
  return torch.triu(torch.ones(mask_size, mask_size) * float('-inf'), diagonal=1)

## 位置エンコーディング
* シーケンス内でのトークンの絶対的な位置をベクトルで表現する。
* 参考資料
  * https://cvml-expertguide.net/terms/dl/seq2seq-translation/transformer/positional-encoding/

In [ ]:
class PositionalEncoding(nn.Module):

  def __init__(self, d_model, dropout=0.1, max_len=5000):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout)

    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
    pe = torch.zeros(max_len, 1, d_model)
    pe[:,0,0::2] = torch.sin(position * div_term)
    pe[:,0,1::2] = torch.cos(position * div_term)
    # `register_buffer()`を使ってpeをこのモジュールのパラメータの一部にする。
    self.register_buffer('pe', pe)

  def forward(self, x):
    # テンソルxの形は[seq_len, batch_size, embedding_dim]
    x = x + self.pe[:x.size(0)]
    return self.dropout(x)

## トークナイザ

* 今回はGPT-2のトークナイザを使う。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
tokenizer.tokenize("This is a pen.")

In [ ]:
tokenizer("This is a pen.")

In [ ]:
tokenizer.vocab_size

### トークン化をおこなうヘルパ関数

* テキスト集合から、一つの長いトークンIDの列を作る。
  * 最大でも1000万トークンまでしか読まないことにする。

In [ ]:
from tqdm.notebook import tqdm

def data_process(raw_text_iter, length=10000000):
  ids = list()
  for item in tqdm(raw_text_iter):
    ids += tokenizer(item.strip())['input_ids']
    if len(ids) > length:
      break
  return torch.tensor(ids[:length], dtype=torch.long)

## データセット




### ``torchtext``を使ったWikitext-2データセットの読み込み

In [ ]:
from torchtext.datasets import WikiText103

train_iter, val_iter, test_iter = WikiText103()

### データセットのトークン化

In [ ]:
train_token_ids = data_process(train_iter)
val_token_ids = data_process(val_iter)
test_token_ids = data_process(test_iter)

* データセットはトークンIDの長い列として表されている。

In [ ]:
train_token_ids.shape, val_token_ids.shape, test_token_ids.shape

In [ ]:
print(tokenizer.convert_ids_to_tokens(train_token_ids[:50]))

In [ ]:
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(train_token_ids[:50])))

## データセットのミニバッチ化

* トークン列を固定長の列に切り分けるヘルパ関数を定義する。
  * トークン列を、バッチサイズの本数に分割する。
  * ただし、分割されたトークン列は、縦向きに並べる。

\begin{align}\begin{bmatrix}
  \text{A} & \text{B} & \text{C} & ... & \text{X} & \text{Y} & \text{Z}
  \end{bmatrix}
  \Rightarrow
  \begin{bmatrix}
  \begin{bmatrix}\text{A} \\ \text{B} \\ \text{C} \\ \text{D} \\ \text{E} \\ \text{F}\end{bmatrix} &
  \begin{bmatrix}\text{G} \\ \text{H} \\ \text{I} \\ \text{J} \\ \text{K} \\ \text{L}\end{bmatrix} &
  \begin{bmatrix}\text{M} \\ \text{N} \\ \text{O} \\ \text{P} \\ \text{Q} \\ \text{R}\end{bmatrix} &
  \begin{bmatrix}\text{S} \\ \text{T} \\ \text{U} \\ \text{V} \\ \text{W} \\ \text{X}\end{bmatrix}
  \end{bmatrix}\end{align}
* これを後で縦方向に細かく分割する。
  * shapeが[入力シーケンス長, バッチサイズ]のミニバッチが得られる。

In [ ]:
def batchify(data, batch_size):
  data_len = data.size(0)
  # 割り切れずに余る分の末尾のトークンは捨てる。
  full_seq_len = data_len // batch_size
  data = data[:full_seq_len * batch_size]
  # `t()`は転置をとる操作
  data = data.reshape(batch_size, full_seq_len).t()
  return data.to(device)

* データセットをミニバッチ化する。

In [ ]:
batch_size = 16
eval_batch_size = 16
train_data = batchify(train_token_ids, batch_size)
val_data = batchify(val_token_ids, eval_batch_size)
test_data = batchify(test_token_ids, eval_batch_size)

In [ ]:
train_data.shape

In [ ]:
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(train_data[:50,0])))

### 入力列とターゲットのペアの作成




* ``get_batch()`` は、入力列とターゲットのペアを作る関数。
* 変数``bptt``で指定された長さの短い列に、元のトークン列を分割する。
* ターゲットは、一次元に潰しておく。
 * 損失関数の計算にはこの方が都合がいいため。



In [ ]:
# シーケンスの最大長
max_seq_len = 128

def get_batch(source, i):
  # i はオフセットを表す。
  # sourceの形は[full_seq_len, batch_size]
  # dataの形は[max_seq_len, batch_size]
  # targetの形は[max_seq_len * batch_size]
  seq_len = min(max_seq_len, len(source) - 1 - i)
  data = source[i:i+seq_len]
  target = source[i+1:i+1+seq_len]
  return data, target

## モデルの作成




In [ ]:
vocab_size = tokenizer.vocab_size  # 語彙サイズ
embed_size = 256  # トークンembeddingの次元
hidden_dim = 256  # nn.TransformerEncoderの隠れ層のサイズ
n_layers = 2  # nn.TransformerEncoderLayerの層の数
n_head = 2  # nn.MultiheadAttentionのヘッドの数
dropout = 0.1  # dropoutの確率
model = TransformerModel(vocab_size, embed_size, n_head, hidden_dim, n_layers, dropout).to(device)

## モデルの訓練



### 損失関数と最適化アルゴリズム

In [ ]:
criterion = nn.CrossEntropyLoss()
lr = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

### 訓練のためのヘルパ関数

In [ ]:
def train(model):
  model.train()  # 訓練モード
  total_loss = 0.
  log_interval = 200
  start_time = time.time()
  src_mask = generate_square_subsequent_mask(max_seq_len).to(device)

  num_batches = len(train_data) // max_seq_len
  full_seq_len = len(train_data)
  for batch, i in enumerate(range(0, full_seq_len - 1, max_seq_len)):
    data, target = get_batch(train_data, i)
    seq_len = data.size(0)
    if seq_len != max_seq_len:  # 最後のミニバッチだけ長さが短い
      src_mask = src_mask[:seq_len, :seq_len]
    output = model(data, src_mask)
    loss = criterion(output.reshape(-1, vocab_size), target.reshape(-1))

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()

    total_loss += loss.item()
    if batch % log_interval == 0 and batch > 0:
      lr = scheduler.get_last_lr()[0]
      ms_per_batch = (time.time() - start_time) * 1000 / log_interval
      cur_loss = total_loss / log_interval
      ppl = math.exp(cur_loss)
      print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
            f'lr {lr:.3e} | ms/batch {ms_per_batch:5.2f} | '
            f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
      total_loss = 0
      start_time = time.time()

### 評価のためのヘルパ関数

In [ ]:
def evaluate(model, eval_data):
  model.eval()  # 評価モード
  total_loss = 0.
  src_mask = generate_square_subsequent_mask(max_seq_len).to(device)
  with torch.no_grad():
    for i in range(0, eval_data.size(0) - 1, max_seq_len):
      data, target = get_batch(eval_data, i)
      seq_len = data.size(0)
      if seq_len != max_seq_len:
        src_mask = src_mask[:seq_len, :seq_len]
      output = model(data, src_mask)
      loss = criterion(output.reshape(-1, vocab_size), target.reshape(-1))
      total_loss += seq_len * loss.item()
  return total_loss / (len(eval_data) - 1)

### 学習の実行


* モデルを保存するパスの設定

In [ ]:
working_directory = os.getcwd() # ここを自分のGoogle Driveのフォルダに変更
best_model_params_path = os.path.join(working_directory, "best_model_params.pt")
print(f"save path: {best_model_params_path}")

* trainingのループを動かす。

In [ ]:
best_val_loss = float('inf')
epochs = 3
best_model = model

for epoch in range(1, epochs + 1):
  epoch_start_time = time.time()
  train(model)
  val_loss = evaluate(model, val_data)
  val_ppl = math.exp(val_loss)
  elapsed = time.time() - epoch_start_time
  print('-' * 89)
  print(
      f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
      f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}'
      )
  print('-' * 89)

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(model.state_dict(), best_model_params_path)

  scheduler.step()

## テストセット上での評価




In [ ]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(
    f'| End of training | test loss {test_loss:5.2f} | '
    f'test ppl {test_ppl:8.2f}'
    )
print('=' * 89)

## テキストの生成

In [ ]:
text = "I couldn't sleep last night. Because I was"
token_ids = torch.tensor([tokenizer(text)["input_ids"]], dtype=torch.int).to(device)
src_mask = generate_square_subsequent_mask(len(token_ids)).to(device)
output = model(token_ids, src_mask)

In [ ]:
output.shape

In [ ]:
output[0,-1,:].argmax()

In [ ]:
tokenizer.convert_ids_to_tokens(output[0,-1,:].argmax().reshape(-1))

In [ ]:
for _ in range(10):
  token_ids = torch.cat([token_ids, output[0,-1,:].argmax().reshape(1,-1)], dim=1)
  src_mask = generate_square_subsequent_mask(len(token_ids)).to(device)
  output = model(token_ids, src_mask)
  print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(token_ids.reshape(-1))))

# 本日の課題
* 最低限、上のコードの動作確認をしよう。
* 余裕があれば、validation perplexityの値をどこまで減らせるか、チューニングしてみよう。